In [ ]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
import matplotlib.pyplot as plt

# Define the path to your dataset folder
dataset_path="C:/Users/aishi/OneDrive/Desktop/Aishik/proj/test 4"

# Define the confidence threshold for labeling (90%)
confidence_threshold = 0.83

# Load your trained CNN model (modify the model loading code accordingly)
model = load_model("terrrain_classification_model.h5")

# Define a mapping of class IDs to labels
class_labels = {
    0: "Water",
    1: "FOREST",
    2: "Sandy"
}

# Function to classify and label a sub-segment
def classify_and_label(sub_image):
    # Resize and preprocess the sub-image
    sub_image = cv2.resize(sub_image, (224, 224))
    sub_image = sub_image / 255.0  # Normalize pixel values

    # Make a prediction using the model
    prediction = model.predict(np.expand_dims(sub_image, axis=0))
    class_probabilities = prediction[0] * 100  # Convert to percentage
    class_id = np.argmax(prediction)

    # Get the class label based on the predicted class ID
    class_label = class_labels[class_id]

    return class_label, class_probabilities

# Function to divide and label sub-segments in the image
def divide_and_label(image):
    height, width, _ = image.shape
    step_size = 100  # Adjust the step size as needed to control sub-segment size

    for y in range(0, height, step_size):
        for x in range(0, width, step_size):
            sub_image =  image[y:y+step_size, x:x+step_size]

            # Ensure the sub-image size is within bounds
            sub_height, sub_width, _ = sub_image.shape
            if sub_height > 0 and sub_width > 0:
                class_label, class_probabilities = classify_and_label(sub_image)

                # Display accuracy predicted for each class within the sub-segment
                print(f"Sub-Segment at ({x}, {y}) - Class: {class_label}")
                for class_id, probability in enumerate(class_probabilities):
                    class_name = class_labels[class_id]
                    print(f"{class_name}: {probability:.2f}%")

                # Draw a bounding box around the sub-segment
                cv2.rectangle(image, (x, y), (x+step_size, y+step_size), (0, 255, 0), 2)
                cv2.putText(image, class_label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

# Load and process an image from the dataset folder
image_file = "C:/Users/aishi/OneDrive/Pictures/Screenshots/gujarat.png"  # Replace with your image file name
image_path = os.path.join(dataset_path, image_file)

if os.path.isfile(image_path):
    # Load the image
    segmented_image = cv2.imread(image_path)

    # Create a copy of the original image for visualization
    visualization_image = segmented_image.copy()

    # Initialize the image labeling process
    divide_and_label(visualization_image)

    # Display the labeled image using matplotlib
    plt.imshow(cv2.cvtColor(visualization_image, cv2.COLOR_BGR2RGB))
    plt.axis('off')
    plt.show()
else:
    print(f"Image not found at path: {image_path}")
